In [ ]:
#import all packages 
import pandas as pd
import warnings
import statsmodels.api as sm
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
import statsmodels.api as sm
import seaborn as sns
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')
from sklearn.preprocessing import OneHotEncoder

data = pd.read_csv('../data/kc_house_data.csv')

In [ ]:
data.corr()

In [ ]:
# drop columns that have no desire to our stakeholder 
clean_null = data.drop(columns=['id','date','long','yr_built','condition','view', 'waterfront',], axis=1)


In [ ]:
clean_null.describe()

In [ ]:
# locating outliers in bedroom 
clean_null = clean_null.loc[clean_null['bedrooms'] < 8] # TODO add comments justifying these
clean_null = clean_null.loc[clean_null['bathrooms'] < 6] 
clean_null = clean_null.loc[clean_null['sqft_living'] < 6000 ]
clean_null = clean_null.loc[clean_null['price'] < 1500000 ]
clean_null = clean_null.loc[clean_null['bedrooms'] > .99]
clean_null = clean_null.loc[clean_null['bathrooms'] > .99]
clean_null = clean_null.loc[clean_null['sqft_living'] > 600 ]
clean_null = clean_null.loc[clean_null['price'] > 100000 ]
clean_null = clean_null.loc[clean_null['sqft_lot'] < 100000 ]
clean_null = clean_null.loc[clean_null['floors'] < 2.1 ]



In [ ]:
clean_null = clean_null.loc[clean_null['sqft_basement'] != '?' ] 

In [ ]:
clean_null['sqft_basement'] = clean_null['sqft_basement'].astype(float)

In [ ]:
clean_null['true_sqft'] = clean_null['sqft_living']  - clean_null['sqft_basement']


In [ ]:
# creating a new feature of price per sqft
clean_null['price_per_sqft'] = clean_null['price']  / clean_null['true_sqft']


In [ ]:
test = [i[0:2].strip() for i in clean_null['grade']]


In [ ]:
clean_null['grade_ordinal'] = [int(i[0:2].strip()) for i in clean_null['grade']]

In [ ]:
zip_home_size = clean_null.loc[clean_null['zipcode'].isin([98052,98102,98103,98105,98109,98119,98121,98134,98164])]



In [ ]:
curated_zip = np.array([98052,98102,98103,98105,98109,98119,98121,98134,98164])

In [ ]:

# TODO add these OHE'd columns to main data by assigning <dfname>['zipcodes_ohe']
#
# take out the isin function for all zipcodes, if that makes the OLS work. could also narrow down to curated after the OHE on all zipcodes
zips = clean_null.loc[clean_null['zipcode'].isin(curated_zip)].dropna() 
ohe = OneHotEncoder(sparse=False, drop=[98109]) # the zip code with the highest prices in Seattle. could do 98052, near the Microsoft campus as well
ohe.fit(zips['zipcode'].to_numpy().reshape(-1, 1))
zips_encoded = ohe.transform(zips['zipcode'].to_numpy().reshape(-1, 1))

zipcodes_ohe = pd.DataFrame(zips_encoded, columns=ohe.get_feature_names(), index=zips.index)

In [ ]:
zip_home_size.columns

In [ ]:
zip_home_size.corr()

In [ ]:
zip_new_columns = zip_home_size.drop(columns=['zipcode','grade','price_per_sqft','sqft_lot15','sqft_lot'], axis=1)
# zip_new_columns.reset_index(inplace=True, drop=True)
zip_new_columns.columns

In [ ]:
X = zip_new_columns #.drop(['price'], axis=1)
ss = StandardScaler().fit(X)
znc_scaled = pd.DataFrame(ss.transform(X), columns=X.columns).dropna() # adding standard scalar to only zip_new_columns, abbreviated as znc

In [ ]:
# concatenating two dataframes, znc_scaled and the one hot encoded zipcodes zipcodes_ohe 
zip_ohe_columns = pd.merge(znc_scaled , zipcodes_ohe, how='outer', left_index=True, right_on=znc_scaled.index) #pd.concat([znc_scaled , zipcodes_ohe], axis=1) 
zip_ohe_columns = zip_ohe_columns.drop('key_0', axis=1)
zip_ohe_columns

In [ ]:
# zip_ohe_columns = zip_ohe_columns.dropna() # don't need to do this because NaNs were dropped above

In [ ]:
#zip_ohe_columns = zip_ohe_columns.drop(columns=['x0_98102',
      # 'x0_98103', 'x0_98105', 'x0_98119'], axis=1)
#zip_new_columns.reset_index(inplace=True, drop=True)

In [ ]:
zip_ohe_columns.corr()

In [ ]:
zip_ohe_columns.columns


In [ ]:

y = zip_ohe_columns['price']
x = zip_ohe_columns['sqft_living']
X_c = sm.add_constant(x)


simple_model = sm.OLS(y, X_c)


simple_model_results = simple_model.fit()
print(simple_model_results.summary())

In [ ]:
#reviewing coef and p value significants 
y = zip_ohe_columns['price']
X = zip_ohe_columns.drop(['price'], axis=1)

y = y.dropna()
X = X.dropna()

# len(y), len(X)
ols_kch = sm.OLS(y, sm.add_constant(X)).fit()
ols_kch.summary()



In [ ]:
#identifying the lots we recommend the stakeholder purchase and build new multifamily homes. 

zip_lot_size = clean_null.loc[(clean_null['zipcode'].isin([98052,98102,98103,98105,98109,98119,98121,98134,98164]))
                              & (clean_null['sqft_lot'] >= 40000)]

zip_lot_size

In [ ]:
x 1 scaled and not scaled (x)
b4 1hot and after 

In [ ]:
#Linearity
# X = zip_ohe_columns.drop(['price'], axis=1)
# ss = StandardScaler().fit(X)

# X_scaled = ss.transform(X)

In [ ]:
X_c = sm.add_constant(X_scaled)
y = zip_ohe_columns['price']

In [ ]:
model = sm.OLS(endog=y, exog=X_c).fit()
model_summary = model.summary()
model_summary

In [ ]:
model_preds = model.predict(X_c)
resids = y - model_preds

fig, ax = plt.subplots()



ax.scatter(model_preds, y)
ax.set_xlabel('predicted home prices')
ax.set_ylabel('residual')
plt.suptitle('Residuals Vs. Predictions');


b, a = np.polyfit(model_preds, y, deg=1)


# Plot regression line
ax.plot(model_preds, a + b * model_preds, color="red", lw=2.5);


In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(8, 6))


ax.scatter(X['bathrooms'], y)
ax.set_title('Home Price Price vs. bath');

In [ ]:
fig, ax = plt.subplots()

ax.scatter(y, resids);

In [ ]:
dw = sm.stats.stattools.durbin_watson

In [ ]:
dw(resids)

In [ ]:
model.summary().tables[2]

In [ ]:
resids.hist(bins=50);

In [ ]:
sm.qqplot(resids, line='r');

In [ ]:
zip_ohe_columns.info()

In [ ]:
zip_ohe_columns['grade_ordinal'].unique()

In [ ]:
fig, ax = plt.subplots()

ax.scatter(y=zip_ohe_columns['price'], x=zip_ohe_columns.drop(['price'], axis=1));

In [ ]:
model.summary().tables[2]

In [ ]:
y.hist();

In [ ]:
fig, ax = plt.subplots()

ax.scatter(y, resids);

In [ ]:
y_log = np.log(y)
y_log.hist();

In [ ]:
# 'x1' is 
sm.graphics.plot_regress_exog(model, 'x1', fig=plt.figure(figsize=(12, 8)));